In [27]:
import numpy as np 
import matplotlib.pyplot as plt
import random as rd
import csv

In [28]:
def ratio_b_over_v(xi):
    return xi[2]/xi[1]

In [29]:
def get_v(solution):
  volume = 0
  for i in range(len(solution)):
    volume += pt[solution[i]-1][1]
  return volume

In [30]:
def get_b(solution):
  benefice = 0
  for i in range(0, len(solution), +1):
    benefice += pt[solution[i]-1][2]
  return benefice

In [31]:
def affichage(solution, nb_element):
  affichage = np.zeros(nb_element, dtype=np.int64)
  for i in range (0, len(solution), +1):
    affichage[solution[i]-1] += 1
  return affichage

In [33]:
def get_voisin(solution, pt, vMax, vMin):
  taille = len(solution)
  nb = np.random.randint(1,taille)
  #print("nb = ",nb)
  for i in range(0, nb, +1):
    x = np.random.randint(0,taille-1)
    #print("supprimer l'élement x =", x)
    del solution[x]
  #print("après suppression")
  #print( solution)
    taille = len(solution)

  volume = get_v(solution)

  #print("maintenant l ajout d'élément")
  while vMax - volume > vMin :
    x = np.random.randint(0,len(pt)-1) 
    #print("l élément à ajouter x =", x)
    #print(str(vMax - volume)+">" +str(pt[x][1]))
    if (vMax - volume > pt[x][1]):
      num_produit = pt[x][0]
      solution.append(num_produit)
      volume += pt[x][1]
  return solution

In [34]:
def get_solution_init(pt):#density order greedy 
  pto = pt.copy()
  pto.sort(key=ratio_b_over_v, reverse=True)
  solution_initiale = [] 
  vAc = 0 #volume actuel
  vMin = 32
  vRes = vMax - vAc
  i = 0
  while (vRes > vMin and i < len(pto)):
    if pto[i][1] < (vMax - vAc):
      num_produit = pto[i][0] 
      solution_initiale.append(num_produit)
      vAc += pto[i][1]
      vRes = vMax - vAc
    else :
      i +=1
  return solution_initiale

In [84]:
def Recuit_simule(pt, max_iter, solution_initiale, temperature_initial, alpha, rnd):
  print("*************paramètres*************")
  print("nombre d'itérations : ", max_iter)
  print("température initiale : ", temperature_initial)
  print("alpha : ", alpha)
  # initialisation de solution courrante
  nb_produits = len(pt)
  (solution_courrante, volume_courrant, benefice_courant) = (solution_initiale, get_v(solution_initiale), get_b(solution_initiale))
  (meilleure_solution, meilleure_benefice) = (solution_courrante, benefice_courant)
  print("\n*********solution initial***********")
  print(affichage(meilleure_solution, nb_produits))
  print("cout initiale : ",meilleure_benefice)
  print("************************************\n")

  temperature_courrante = temperature_initial
  iteration = 0
  interval = (int)(max_iter / 10)
  while iteration < max_iter:
      solution_voisin = get_voisin(solution_courrante, pt, vMax, vMin ) # générer une solution voisine en utilisant la fonction aléatoire 
      (volume_voisin, benefice_voisin) = (get_v(solution_voisin),get_b(solution_voisin))

      if benefice_voisin > benefice_courant:  
        (solution_courrante, volume_courrant, benefice_courant) = (solution_voisin, volume_voisin, benefice_voisin)
        if (meilleure_benefice < benefice_courant):
          (meilleure_solution, meilleure_benefice) = (solution_courrante, benefice_courant)

      else:          # si le cout du voisin est inférieure
        accept_p = np.exp( (benefice_voisin - benefice_courant ) / temperature_courrante ) 
        p = rnd.random()
        if p < accept_p: 
          (solution_courrante, volume_courrant, benefice_courant) = (solution_voisin, volume_voisin, benefice_voisin)
        # else don't accept

      if iteration % interval == 0:
        print("iter = %6d  benefice = %7.0f volume = %7.0f temperature = %10.2f  solution courrante = " % (iteration, benefice_courant, volume_courrant,  temperature_courrante), affichage(solution_courrante, nb_produits))
      if temperature_courrante < 0.00001:
        temperature_courrante = 0.00001
      else:
        temperature_courrante*= alpha

      iteration += 1
  
  print("\n*********meilleure_benefice***********")
  print("meilleure cout : ",meilleure_benefice)
  return meilleure_solution, meilleure_benefice

In [85]:
def load_pt_from_csvF():
    fil = open('knapsac_instance.csv')
    csvreader = csv.reader(fil)
    header=[]
    header=next(csvreader)
    rows = []
    i=1
    pt=[]
    l=[]
    for row in csvreader :
        l = [i, int(row[0]), int(row[1])]
        pt.append(l)
        i=i+1
    return pt

In [86]:
#*******************************test 1***********************************
# volume du sac
vMax = 130
vMin = 32
# produits
x1 = [1,33,4]
x2 = [2,49,5]
x3 = [3,60,6]
x4 = [4,32,2]

# table des produits
pt = [x1,x2,x3,x4]

#génération de la solution initiale
solution_initiale = get_solution_init(pt)

(meilleure_solution, meilleure_benefice)=Recuit_simule(pt, 10, solution_initiale, 100, 0.98,np.random.RandomState(5))



*************paramètres*************
nombre d'itérations :  10
température initiale :  100
alpha :  0.98

*********solution initial***********
[3 0 0 0]
cout initiale :  12
************************************

iter =      0  benefice =      13 volume =     115 temperature =     100.00  solution courrante =  [2 1 0 0]
iter =      1  benefice =      13 volume =     115 temperature =      98.00  solution courrante =  [2 1 0 0]
iter =      2  benefice =      13 volume =     115 temperature =      96.04  solution courrante =  [2 1 0 0]
iter =      3  benefice =      13 volume =     115 temperature =      94.12  solution courrante =  [2 1 0 0]
iter =      4  benefice =      13 volume =     115 temperature =      92.24  solution courrante =  [2 1 0 0]
iter =      5  benefice =      14 volume =     126 temperature =      90.39  solution courrante =  [2 0 1 0]
iter =      6  benefice =      14 volume =     126 temperature =      88.58  solution courrante =  [2 0 1 0]
iter =      7  benefice = 

In [87]:
#*******************************test 2***********************************
# volume du sac
vMax = 500000
vMin = 260
# produits
pt = load_pt_from_csvF()

# table des produits


#génération de la solution initiale
solution_initiale = get_solution_init(pt)

(meilleure_solution, meilleure_benefice)=Recuit_simule(pt, 20, solution_initiale, 100, 0.98,np.random.RandomState(5))


*************paramètres*************
nombre d'itérations :  20
température initiale :  100
alpha :  0.98

*********solution initial***********
[ 0  0  0  0  0 61  0  0  0  0  0  0  0  0  1  0  0  0  1]
cout initiale :  558697
************************************

iter =      0  benefice =  558697 volume =  499742 temperature =     100.00  solution courrante =  [ 4  3  0  6  6 33  6  2  4  2  5  3  7  7  2  6  5  5  1]
iter =      2  benefice =  558697 volume =  499742 temperature =      96.04  solution courrante =  [ 6  5  4  5 12 17  8  5  7  4 10  4  7  7 11  5  6  9  0]
iter =      4  benefice =  558697 volume =  499742 temperature =      92.24  solution courrante =  [11  4  4  8  7 11  8  7  7  6 11  7 12 11 12  3  8 12  0]
iter =      6  benefice =  558697 volume =  499742 temperature =      88.58  solution courrante =  [ 7  5  6  6  8  5  6  9 11 10 17  2 11 12  8  7  8  8  0]
iter =      8  benefice =  558697 volume =  499742 temperature =      85.08  solution courrante =  [10  